In [6]:
import requests
from bs4 import BeautifulSoup
import re

def scrape_colleges(city=None, state=None):
    colleges = []
    
    if city:
        url = f"https://www.google.com/search?q=colleges+in+{city}&num=100"
    elif state:
        url = f"https://en.wikipedia.org/wiki/List_of_institutions_of_higher_education_in_+{state}"
    else:
        url = "https://en.wikipedia.org/wiki/List_of_cities_in_India_by_population"
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('table', class_='wikitable sortable')
        rows = table.find_all('tr')[1:]
        cities = [row.find_all('td')[1].text.strip() for row in rows]
        
        for city in cities:
            colleges.extend(scrape_colleges(city=city))
        
        return colleges
    
    html = requests.get(url).content
    soup = BeautifulSoup(html, 'html.parser')
    
    if city:
        results = soup.find_all('div', class_='BNeawe s3v9rd AP7Wnd')
    else:
        results = soup.find_all('a', class_='mw-redirect')
    
    for result in results:
        link = result.find('a')['href']
        name = result.text.strip()
        
        try:
            html = requests.get(link).content
            soup = BeautifulSoup(html, 'html.parser')
            
            address = soup.find('span', {'itemprop': 'address'}).text.strip()
        except:
            address = None
            
        try:
            email = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', html.decode())[0]
        except:
            email = None
            
        try:
            phone = re.findall(r'[\d+ -]{10,}', html.decode())[0]
        except:
            phone = None
        
        colleges.append({'name': name, 'link': link, 'address': address, 'email': email, 'phone': phone})
    
    return colleges

# Get user input
print("Enter 1 for tire 1 city=Mumbai,Delhi, kalkatta, Chennai")
print("Enter 2= Capital of every state")
print("Enter 3= Total city of every state")
option = int(input())

if option == 1:
    city = input("Enter city name: ")
    colleges = scrape_colleges(city=city)
elif option == 2:
    state = input("Enter state name: ")
    colleges = scrape_colleges(state=state)
elif option == 3:
    colleges = scrape_colleges()
else:
    print("Invalid option")
    exit()

# Generate report
print(f"Number of colleges found: {len(colleges)}\n")

for college in colleges:
    print(college['name'])
    print(college['link'])
    print(college['address'])
    print(college['email'])
    print(college['phone'])
    print("\n")

Enter 1 for tire 1 city=Mumbai,Delhi, kalkatta, Chennai
Enter 2= Capital of every state
Enter 3= Total city of every state
2
Enter state name: Maharashtra
Number of colleges found: 0

